# Version suitable to our case

The problem in the implementation above is, that training and testing data uses the electric power. Obviously they provide good results. However, our task where we want to model the behavior depending on the input beforehand is not solved.

For our task we need to implement a network where it takes the current and previous inputs to predict the electrical output. If useful, we can include the previous electrical oputput as well.

For all the data we need to implement the difference, scale all and then train the model.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import Global_Functions as gf
import matplotlib.pyplot as plt
import Neuronal_Networks as nn

In [2]:
import time
timestr = time.strftime("%Y-%m-%d_%H-%M_")

In [3]:
OPEN_FOLDER = '../Data/Preped_Data/'

In [4]:
ex_1 = gf.open_CSV_file('experiment_1_short.csv', OPEN_FOLDER)
ex_4 = gf.open_CSV_file('experiment_4_short.csv', OPEN_FOLDER)
ex_9 = gf.open_CSV_file('experiment_9_short.csv', OPEN_FOLDER)
ex_20 = gf.open_CSV_file('experiment_20_short.csv', OPEN_FOLDER)
ex_21 = gf.open_CSV_file('experiment_21_short.csv', OPEN_FOLDER)
ex_22 = gf.open_CSV_file('experiment_22_short.csv', OPEN_FOLDER)
ex_23 = gf.open_CSV_file('experiment_23_short.csv', OPEN_FOLDER)
ex_24 = gf.open_CSV_file('experiment_24_short.csv', OPEN_FOLDER)

In [5]:
train = "4"
val = "21"

ex_train = ex_4
ex_val = ex_21

In [6]:
NEUR = 128
EPOCH = 100
LAG = 60

In [7]:
image_path = "../Images/"
image_subfolder = image_path + "RNN/"
specs = "training_{0} val_{1} {2}_neurons__{3}_epochs__lag_{4}" 
image_folder = image_subfolder + timestr + specs.format(train, val, NEUR, EPOCH, LAG) + "/"

gf.check_folder(image_path)
gf.check_folder(image_subfolder)
gf.check_folder(image_folder)

Folder already exists.
Folder already exists.
Creation of directory ../Images/RNN/2021-03-19_09-36_training_4 val_21 128_neurons__100_epochs__lag_60/ successful.


In [8]:
np.random.seed(123)

In [9]:
kit_orange = gf.get_color('orange')
kit_green = gf.get_color('green')
kit_blue = gf.get_color('blue')
kit_cyan = gf.get_color('cyan')
kit_red = gf.get_color('red')
kit_lightgreen = gf.get_color('lightgreen')
kit_grey = gf.get_color('grey')

## Make time series to supervised problem

We need to restructure the data to have a supervised learning problem. Within we need to get the necessary inputs and necessary outputs as new columns.

In [10]:
# def timeseries_to_supervised(data, lag = 10):
#     df = pd.DataFrame(data)
#     columns = [df.shift(i) for i in range(1, lag +1)]
#     df_delay = pd.concat(columns, axis = 1)
#     df = pd.concat([df, df_delay], axis = 1)
#     df.fillna(0, inplace = True)
#     df.columns = [str("delay_" + str(i)) for i in range(0, lag +1)]
#     return df

In [11]:
# def difference(data, lag = 10):
#     diff = list()
#     for i in range(lag, len(data)):
#         value = data[i] - data[i - lag]
#         diff.append(value)
#     return pd.Series(diff, name = data.name)

In [12]:
# def invert_difference(data, yhat, index = 0):
#     return yhat + data[index]

In [13]:
# def scale(data_series, scaler = None):
#     data = data_series.values
#     data = data.reshape(-1, 1)
#     if scaler is None:
#         scaler = MinMaxScaler(feature_range = (-1,1), )
#         scaler = scaler.fit(data)
#     data_scaled = scaler.transform(data)
#     return data_scaled, scaler

In [14]:
# def invert_scale(scaler, value):
#     return scaler.inverse_transform(value.reshape(-1,1))

In [15]:
# def fit_lstm(X_train, y_train, X_val, y_val, batch_size, nb_epochs, neurons): 
#     model = Sequential()
#     model.add(layers.LSTM(neurons, batch_input_shape = (batch_size, X_train.shape[1], X_train.shape[2]), stateful = True))
#     model.add(layers.Dense(1))
#     model.compile(loss = 'mean_squared_error', optimizer = 'adam', )
#     for i in range(nb_epochs):
#         model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = 1, batch_size = batch_size, verbose = 1, shuffle = False)
#         model.reset_states()
#     return model

## Second try

We do all from scratch building our own data frame and using scaling and preparation of data according to our understanding

In [16]:
def train_model(experiment_train, experiment_val, lag_input = 1, lag_output = 1, batch_size = 1, neurons = 8, nb_epochs = 10):
    scaler_train, X_train, y_train = nn.prepare_data(experiment_train, lag_input, lag_output, all_lags = False)
    scaler_val, X_val, y_val = nn.prepare_data(experiment_val, lag_input, lag_output, all_lags = False)
    
    model = nn.fit_lstm(X_train, y_train, X_val, y_val, batch_size, nb_epochs, neurons)
    
    return model, scaler_train, X_train, y_train, scaler_val, X_val, y_val

In [ ]:
model, scaler, X_train, y_train, _, X_val, y_val = train_model(ex_train, ex_val, lag_input=LAG, lag_output = 1, batch_size=1, neurons=NEUR, nb_epochs=EPOCH)

9763/9795 [============================>.] - ETA: 0s - loss: 0.0062

In [ ]:
def predictions(experiment, model, batch_size = 1, specs = ""):
    scaler, X, y = nn.prepare_data(experiment, lag_input=LAG, all_lags = False)
    
    preds_scaled = model.predict(X, batch_size = batch_size)
    preds = scaler[1].inverse_transform(preds_scaled)
    
    fig = plt.figure(figsize = (15,10))
    plt.plot(experiment['el_power'], color = gf.get_color("grey"), label = "True")
    plt.plot(preds, color = gf.get_color("green"), label = "Predictions")
    plt.ylabel('Electric power in [W]')
    plt.xlabel('Time')
    plt.ylim([0, 4000])
    plt.legend()
    plt.title('Comparison predictions to true values' + specs, fontsize = 14)
    fig.tight_layout()
    plt.show()
    fig.savefig(image_folder + specs + "predictions.png")
    fig.savefig(image_folder + specs + "predictions.svg")
    
    return scaler, X, y, preds_scaled, preds

In [ ]:
scaler_1, X_1, y_1, preds_scaled_1, preds_1 = predictions(ex_1, model, specs = " on ex_1 with model trained on " + train)

In [ ]:
scaler_4, X_4, y_4, preds_scaled_4, preds_4 = predictions(ex_4, model, specs = " on ex_4 with model trained on " + train)

In [ ]:
scaler_9, X_9, y_9, preds_scaled_9, preds_9 = predictions(ex_9, model, specs = " on ex_9 with model trained on " + train)

In [ ]:
scaler_20, X_20, y_20, preds_scaled_20, preds_20 = predictions(ex_20, model, specs = " on ex_20 with model trained on " + train)

In [ ]:
scaler_21, X_21, y_21, preds_scaled_21, preds_21 = predictions(ex_21, model, specs = " on ex_21 with model trained on " + train)

In [ ]:
scaler_22, X_22, y_22, preds_scaled_22, preds_22 = predictions(ex_22, model, specs = " on ex_22 with model trained on " + train)

In [ ]:
scaler_23, X_23, y_23, preds_scaled_23, preds_23 = predictions(ex_23, model, specs = " on ex_23 with model trained on " + train)

In [ ]:
scaler_24, X_24, y_24, preds_scaled_24, preds_24 = predictions(ex_24, model, specs = " on ex_24 with model trained on " + train)